In [32]:
#import dependcies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc
from sqlalchemy.sql import label

from flask import Flask, jsonify

In [33]:
#set up db
engine = create_engine("sqlite:///hawaii.sqlite")

In [34]:
#reflect db
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [35]:
#reference tables
Measurement = Base.classes.measurements
Stations = Base.classes.stations

AttributeError: stations

In [36]:
#create session
session = Session(bind = engine)

In [ ]:
#get table names
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
#get column names & types for measurements
columns = inspector.get_columns('measurements')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
#get column names & types for stations
columns = inspector.get_columns('stations')
for c in columns:
    print(c['name'], c["type"])

In [37]:
#Climate Analysis
#query last year of prcp data
results = session.query(Measurement.date, Measurement.precip).filter(Measurement.date >= '2017-01-01')

AttributeError: type object 'measurements' has no attribute 'precip'

In [38]:
#create date & prcp df
data = {'date': [], 'precip': []}

for row in results:
    data['date'].append(row.date)
    data['precip'].append(row.precip)

# create pandas df from dict
prcp_df = pd.DataFrame(data['precip'], columns = ['prcp'], index = data['date'])

NameError: name 'results' is not defined

In [ ]:
prcp_df.head()

In [ ]:
plt.style.use('ggplot')
fig = plt.figure()

prcp_df.plot(figsize = (18,8), color='blue', rot = 340 )
plt.show()

In [ ]:
#Station Analysis
total_stations = session.query(func.count(Stations.station)).all()

In [ ]:
#most active stations
active_stations = session.query(measurement.station, station.name, measurement.date, func.count(measurement.tobs)).\
                          join(station, station.station == measurement.station).\
                          group_by(measurement.station).\
                          order_by(func.count(measurement.tobs).desc()).all()


In [44]:
#station with most observations
most_observations = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).first()

In [46]:
#create df for 12 bins plot
df = pd.DataFrame(temps, columns=['Station', 'date', 'temp'])
df.set_index('Station', inplace=True)
df.head()

NameError: name 'temps' is not defined

In [47]:
#histogram plot
hist_plot = df['temp'].hist(bins=12, figsize=(15,10))
hist_plot.set_title('Temperature Observations', fontsize=20)
hist_plot.set_ylabel('Frequency', fontsize=20)
hist_plot.set_ylabel('Temperature', fontsize=20)
plt.show()

NameError: name 'df' is not defined

In [ ]:
#temp analysis
def calc_temps(start_date, end_date):
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

In [ ]:
#create df for plot
temperatures = (calc_temps('2017-05-28', '2017-06-03'))
labels = ['TMIN', 'TAVE', 'TMAX']
temp_df = pd.DataFrame.from_records(temperatures, columns=labels)

In [ ]:
#create bar chart 
temp_chart = df[['TAVE']].plot(kind='bar', title ="Temperature in Hawaii", figsize=(5, 7), legend=True, fontsize=12, grid=True, color='pink')
temp_chart.set_xlabel("Temprature Avg", fontsize=12)
temp_chart.set_ylabel("Temperatures in Farenheit", fontsize=12)
plt.show()

In [ ]:
#Flask- climate app
